<a href="https://colab.research.google.com/github/kimsejin111/speedlimit_classification/blob/main/speedlimits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import PIL
from PIL import Image
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import pathlib

In [ ]:
data_dir='/content/gdrive/MyDrive/data'

In [ ]:
data_dir=pathlib.Path(data_dir)

batch_size=32
image_height=180
image_width=180

d_image_count = len(list(data_dir.glob('*/*.png')))
print(d_image_count)


In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2, # 왜 0.2로 하는거지?
    subset="training",
    seed=123,   # seed는 무얼 의미하는 거지?
    image_size=(image_height, image_width),
    batch_size=batch_size
)
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(image_height, image_width),
    batch_size=batch_size
)

In [ ]:
class_names = train_ds.class_names
plt.figure(figsize=(10,10))
for images, labels in train_ds.take(1): #take 함수는 무슨 함수일까
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8")) #numpy,imshow 는 무슨 함수일까
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()